In [ ]:
import mne
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import confusion_matrix
from mne.decoding import CSP
from sklearn.svm import SVC
import random
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, precision_score, recall_score, f1_score


import pylab
import matplotlib.ticker as ticker
%matplotlib inline

**Table 2**

**Model trained** on Clean trials from tradtional RSVP

**Model tested** on clean, bad and combination of clean & bad trials from each tradtional as well as Intentionally Contaminated IC-RSVP

In [ ]:
import csv
THR=100e-6 # P2P threshold for trial rejection 
# Open a CSV file for writing
with open('RocAuc_TrainClean_100uv.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['P_Num', 'RSVP-All', 'RSVP-Clean','RSVP-Rejected', 'XXXXXXXXXXX', 'Body-All', 'Body-Clean','Body-Rejected', 'XXXXXXXXXXX',
                     
                     'Talk-All', 'Talk-Clean','Talk-Rejected', 'XXXXXXXXXXX', 'Head-All', 'Head-Clean','Head-Rejected'])

    for i in range(1, 11):
        P_Num = i
        
        
        
        #P_Num=10

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        def process_file(edf_file_path, csv_file_path, filename):
            raw=mne.io.read_raw_edf(edf_file_path, preload=True)

            # Pick 6 out of 32 channels
            #channel_names_to_select = ['Fz-CPz','Cz-CPz', 'P3-CPz', 'Pz-CPz', 'P4-CPz','Oz-CPz']
            #raw.pick_channels(channel_names_to_select)

            #Applying lowpass filter at 30Hz
            raw = raw.filter(.1, 30, method='iir')

            # Resampling data from 1000sps to 100sps
            raw = raw.resample(sfreq=100)


            #Re-Referencing the data to common average reference (CAR)
            raw= raw.set_eeg_reference(ref_channels='average')


            data = raw.get_data()
            Samples=len(data[0])

            annt=mne.events_from_annotations(raw, event_id='auto', regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$', use_rounding=True, chunk_duration=None, verbose=None)
            ann0=annt[0]
            Markers_Raw=ann0[:,0]
            L_annt_r=len(Markers_Raw)

            if (L_annt_r == 362):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end


            # Subject 5 -- Session 4 -- X1 :-
            ## This process is described and done in detail in another notebook.
            # there are total 371 markers instead of 362.
            # 1 marker from start and 9 markers from end are extra markers.
            # 1 Marker in at index 185 is also extra. We will remove it.

            if (L_annt_r == 371):
                Markers=Markers_Raw[1:-9] # :- Extras = 1 start and 9 end
                index1 = 185 # index_to_remove
                Markers = np.delete(Markers, index1)

            # Subject 7 -- Session 1 -- X1 :-
            ## This process is described and done in detail in another notebook.
            # there are total 371 markers instead of 362.
            # 1 marker from start and 1 marker from end are extra markers.
            # 2 Markers at index 116 and index 180 are missing. We will insert these markers.


            if (L_annt_r == 360):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end

                index1 = 116
                value1 = Markers[index1 - 1] + 25

                index2 = 180
                value2 = Markers[index2 - 1] + 25

                insert_indices = [index1, index2]
                values_to_insert = [value1, value2]

                Markers = np.insert(Markers, insert_indices, values_to_insert)    

            # Subject 8 -- Session 1 -- X4 :-
            ## This process is described and done in detail in another notebook.
            # there are total 363 markers instead of 362.
            # 1 marker from start and 9 markers from end are extra markers.
            # 1 Marker in at index 200 is also extra. We will remove it.

            if (L_annt_r == 363):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end
                index1 = 200 # index_to_remove
                Markers = np.delete(Markers, index1)

            # Subject 7 -- Session 1 -- X8 :-
            ## This process is described and done in detail in another notebook.
            # there are total 359 markers instead of 362.
            # 1 marker from start and 1 marker from end are extra markers.
            # 3 Markers at index 49, 50 and index 139 are missing. We will insert these markers.


            if (L_annt_r == 359):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end

                index1 = 49
                value1 = Markers[index1 - 1] + 25

                index2 = 50
                value2 = Markers[index2 - 1] + 25

                index3 = 139
                value3 = Markers[index3 - 1] + 25

                insert_indices = [index1, index2, index3]
                values_to_insert = [value1, value2, value3]

                Markers = np.insert(Markers, insert_indices, values_to_insert)     



            else:
                print("Unknown length")
            L_annt0=len(Markers)
            print("Number of markers: ",L_annt0)

            Zeros = [0] * Samples
            Zero_arr = np.array(Zeros)
            Zero_2d = np.reshape(Zero_arr, (len(Zero_arr), 1))
            Zero_2dt=np.transpose(Zero_2d)

            #csv_path= file_path.replace('Data', 'Markers').replace('.edf', '.csv')
            csv_mark0 = pd.read_csv(csv_file_path, header=None)
            csv_mark = np.array(csv_mark0)

            zero_mark=Zero_arr
            csv1=0
            for x in Markers:   
                zero_mark[x]=csv_mark[csv1]
                csv1=csv1+1

            tar = np.where(zero_mark == 1)
            std = np.where(zero_mark == 2)

            new_events = []
            for st in std[0]:
                new_events.append([st, 0, 2]) 
            for t in tar[0]:
                new_events.append([t, 0, 1])

            event_dict = {'target':1, 'standard':2}
            epoch = mne.Epochs(raw, new_events, tmin=-0.2, tmax=0.8, event_id=event_dict,
                            preload=True, baseline=None)

            #----------------------------------------

            evoked_target = epoch['target'].average()
            evoked_standard = epoch['standard'].average()
            evoked_diff = mne.combine_evoked([evoked_target, evoked_standard], weights=[1, -1])
            #----------------------------------------------

     
            return epoch
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        # Only RSVP Tasks ( X1 and X2 )
        # Clean ---> c
        # There are two X1s and two X2s in each session

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_c1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X1-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_c1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X1-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c1 = os.path.basename(edf_path_c1).split('.')[0]
        epoch_c1 = process_file(edf_path_c1,csv_path_c1, filename_c1)



        edf_path_c2 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c2 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c2 = os.path.basename(edf_path_c2).split('.')[0]
        epoch_c2 = process_file(edf_path_c2,csv_path_c2, filename_c2)

        #----------------------------------------Session-2----------------------------------------

        edf_path_c3 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c3 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c3 = os.path.basename(edf_path_c3).split('.')[0]
        epoch_c3 = process_file(edf_path_c3,csv_path_c3, filename_c3)

        edf_path_c4 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c4 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c4 = os.path.basename(edf_path_c4).split('.')[0]
        epoch_c4 = process_file(edf_path_c4,csv_path_c4, filename_c4)

        #----------------------------------------Session-3----------------------------------------

        edf_path_c5 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c5 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c5 = os.path.basename(edf_path_c5).split('.')[0]
        epoch_c5 = process_file(edf_path_c5,csv_path_c5, filename_c5)

        edf_path_c6 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c6 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c6 = os.path.basename(edf_path_c6).split('.')[0]
        epoch_c6 = process_file(edf_path_c6,csv_path_c6, filename_c6)

        #----------------------------------------Session-4----------------------------------------

        edf_path_c7 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c7 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c7 = os.path.basename(edf_path_c7).split('.')[0]
        epoch_c7 = process_file(edf_path_c7,csv_path_c7, filename_c7)

        edf_path_c8 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c8 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c8 = os.path.basename(edf_path_c8).split('.')[0]
        epoch_c8 = process_file(edf_path_c8,csv_path_c8, filename_c8)
 
        # Batch 1 training purposes
        Epoch_C_Tr = mne.concatenate_epochs([epoch_c1, epoch_c2, epoch_c4, epoch_c5, epoch_c7, epoch_c8])
        Epoch_C_Ts = mne.concatenate_epochs([epoch_c3, epoch_c6])



        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        # RSVP with Body Movement ( X4 )
        # Body ---> b

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_b1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X4-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_b1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X4-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b1 = os.path.basename(edf_path_b1).split('.')[0]
        epoch_b1 = process_file(edf_path_b1,csv_path_b1, filename_b1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_b2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b2 = os.path.basename(edf_path_b2).split('.')[0]
        epoch_b2 = process_file(edf_path_b2,csv_path_b2, filename_b2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_b3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b3 = os.path.basename(edf_path_b3).split('.')[0]
        epoch_b3 = process_file(edf_path_b3,csv_path_b3, filename_b3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_b4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b4 = os.path.basename(edf_path_b4).split('.')[0]
        epoch_b4 = process_file(edf_path_b4,csv_path_b4, filename_b4)

        #------------------
        Epoch_B = mne.concatenate_epochs([epoch_b1, epoch_b2, epoch_b3, epoch_b4])

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        # RSVP with Talking ( X6 )
        # Talking ---> t

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_t1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X6-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_t1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X6-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t1 = os.path.basename(edf_path_t1).split('.')[0]
        epoch_t1 = process_file(edf_path_t1,csv_path_t1, filename_t1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_t2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t2 = os.path.basename(edf_path_t2).split('.')[0]
        epoch_t2 = process_file(edf_path_t2,csv_path_t2, filename_t2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_t3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t3 = os.path.basename(edf_path_t3).split('.')[0]
        epoch_t3 = process_file(edf_path_t3,csv_path_t3, filename_t3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_t4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t4 = os.path.basename(edf_path_t4).split('.')[0]
        epoch_t4 = process_file(edf_path_t4,csv_path_t4, filename_t4)

        #------------------
        Epoch_T = mne.concatenate_epochs([epoch_t1, epoch_t2, epoch_t3 ,epoch_t4])

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        # RSVP with Head Movement ( X8 )
        # Head ---> h

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_h1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X8-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_h1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X8-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h1 = os.path.basename(edf_path_h1).split('.')[0]
        epoch_h1 = process_file(edf_path_h1,csv_path_h1, filename_h1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_h2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h2 = os.path.basename(edf_path_h2).split('.')[0]
        epoch_h2 = process_file(edf_path_h2,csv_path_h2, filename_h2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_h3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h3 = os.path.basename(edf_path_h3).split('.')[0]
        epoch_h3 = process_file(edf_path_h3,csv_path_h3, filename_h3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_h4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h4 = os.path.basename(edf_path_h4).split('.')[0]
        epoch_h4 = process_file(edf_path_h4,csv_path_h4, filename_h4)

        #------------------
        Epoch_H = mne.concatenate_epochs([epoch_h1,epoch_h2, epoch_h3, epoch_h4])
        
        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------       
        #THR=100e-6 #Threshold
        # Set your peak-to-peak threshold
        p2p_threshold_C = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_B = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_T = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_H = THR  # #you can change it.. may be 100e-6 as well

        # Make the copy of epochs
        Epoch_C_All=Epoch_C_Tr.copy()
        Epoch_B_All=Epoch_B.copy()
        Epoch_T_All=Epoch_T.copy()
        Epoch_H_All=Epoch_H.copy()
        
        # Get the data from epochs
        Epoch_C_All_Data = Epoch_C_All.get_data()
        Epoch_B_All_Data = Epoch_B_All.get_data()
        Epoch_T_All_Data = Epoch_T_All.get_data()
        Epoch_H_All_Data = Epoch_H_All.get_data()
    

        # Calculate the peak-to-peak amplitude along the time axis
        p2p_amplitude_C = np.ptp(Epoch_C_All_Data, axis=2)
        p2p_amplitude_B = np.ptp(Epoch_B_All_Data, axis=2)
        p2p_amplitude_T = np.ptp(Epoch_T_All_Data, axis=2)
        p2p_amplitude_H = np.ptp(Epoch_H_All_Data, axis=2)
        

        # Find epochs that exceed the peak-to-peak threshold
        bad_epochs_C = np.where(p2p_amplitude_C > p2p_threshold_C)[0]
        bad_epochs_B = np.where(p2p_amplitude_B > p2p_threshold_B)[0]
        bad_epochs_T = np.where(p2p_amplitude_T > p2p_threshold_T)[0]
        bad_epochs_H = np.where(p2p_amplitude_H > p2p_threshold_H)[0]
        

        # Clean Epochs :- Drop bad epochs from the Epochs object
        Clean_epochs_C = Epoch_C_All.copy()
        Clean_epochs_C.drop(bad_epochs_C)
        
        Clean_epochs_B = Epoch_B_All.copy()
        Clean_epochs_B.drop(bad_epochs_B)
        
        Clean_epochs_T = Epoch_T_All.copy()
        Clean_epochs_T.drop(bad_epochs_T)
        
        Clean_epochs_H = Epoch_H_All.copy()
        Clean_epochs_H.drop(bad_epochs_H)

        # Rejected Epochs 
        
        Clean_indices_C = set(Clean_epochs_C.selection)                # Find the indices of clean epochs
        All_indices_C = set(Epoch_C_All.selection)                     # Find the indices of all epochs
        Rejected_indices_C = All_indices_C - Clean_indices_C           # Find the indices of rejected epochs
        Rejected_indices_C = list(Rejected_indices_C)                  # making a list of indices
        Rejected_indices_C = np.array(Rejected_indices_C).astype(int)  # Converting the list of indices to int
        Rejected_epochs_C = Epoch_C_All[Rejected_indices_C]
        
        Clean_indices_B = set(Clean_epochs_B.selection)                # Find the indices of clean epochs
        All_indices_B = set(Epoch_B_All.selection)                     # Find the indices of all epochs
        Rejected_indices_B = All_indices_B - Clean_indices_B           # Find the indices of rejected epochs
        Rejected_indices_B = list(Rejected_indices_B)                  # making a list of indices
        Rejected_indices_B = np.array(Rejected_indices_B).astype(int)  # Converting the list of indices to int
        Rejected_epochs_B = Epoch_B_All[Rejected_indices_B] 
        
        Clean_indices_T = set(Clean_epochs_T.selection)                # Find the indices of clean epochs
        All_indices_T = set(Epoch_T_All.selection)                     # Find the indices of all epochs
        Rejected_indices_T = All_indices_T - Clean_indices_T           # Find the indices of rejected epochs
        Rejected_indices_T = list(Rejected_indices_T)                  # making a list of indices
        Rejected_indices_T = np.array(Rejected_indices_T).astype(int)  # Converting the list of indices to int
        Rejected_epochs_T = Epoch_T_All[Rejected_indices_T]
        
        Clean_indices_H = set(Clean_epochs_H.selection)                # Find the indices of clean epochs
        All_indices_H = set(Epoch_H_All.selection)                     # Find the indices of all epochs
        Rejected_indices_H = All_indices_H - Clean_indices_H           # Find the indices of rejected epochs
        Rejected_indices_H = list(Rejected_indices_H)                  # making a list of indices
        Rejected_indices_H = np.array(Rejected_indices_H).astype(int)  # Converting the list of indices to int
        Rejected_epochs_H = Epoch_H_All[Rejected_indices_H]
        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------    
        
        # Getting the percentage of dropped/bad epochs

        percentage_dropped_C=(len(Rejected_epochs_C) / len(Epoch_C_All)) * 100
        
        percentage_dropped_B=(len(Rejected_epochs_B) / len(Epoch_B_All)) * 100
        
        percentage_dropped_T=(len(Rejected_epochs_T) / len(Epoch_T_All)) * 100   
        
        percentage_dropped_H=(len(Rejected_epochs_H) / len(Epoch_H_All)) * 100
        
        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------   
        
        # Traditional RSVP Clean Epochs for Training
        Data_C = Clean_epochs_C.get_data()
        Data_C = Data_C.reshape(Data_C.shape[0], -1)
        Events_C = Clean_epochs_C.events[:, -1]

        # Traditional RSVP Test Epochs for Testing (mix-clean and bad) # This set was seperated at start :- Epoch 3 and 6 only
        if percentage_dropped_C < 80 :
            Data_C_All = Epoch_C_Ts.get_data()
            Data_C_All = Data_C_All.reshape(Data_C_All.shape[0], -1)
            Events_C_All = Epoch_C_Ts.events[:, -1]

        else:
            Data_C_All = 0
            Events_C_All = 0
            print("Percentage of dropped epochs-C is greater than 80")
        
        # Traditional RSVP Rejected Epochs for Testing
        if percentage_dropped_C > 0 :
            Data_C_Rejected = Rejected_epochs_C.get_data()
            Data_C_Rejected = Data_C_Rejected.reshape(Data_C_Rejected.shape[0], -1)
            Events_C_Rejected = Rejected_epochs_C.events[:, -1]
        else:
            Data_C_Rejected = 0
            Events_C_Rejected = 0
            print("Percentage of dropped epochs-C is 0")
              
        ##------------------------------------------------------------------------------------------
        
        # RSVP-Body all epochs for testing 
        Data_B_All = Epoch_B_All.get_data()
        Data_B_All = Data_B_All.reshape(Data_B_All.shape[0], -1)
        Events_B_All = Epoch_B_All.events[:, -1]

        # RSVP-Body clean epochs for testing
        if percentage_dropped_B < 80 :
            Data_B_Clean = Clean_epochs_B.get_data()
            Data_B_Clean = Data_B_Clean.reshape(Data_B_Clean.shape[0], -1)
            Events_B_Clean = Clean_epochs_B.events[:, -1]     
        else:
            Data_B_Clean = 0
            Events_B_Clean = 0
            print("Percentage of dropped epochs-B is greater than 80")
            
        # RSVP-Body rejected epochs for testing
        if percentage_dropped_B > 0 :
            Data_B_Rejected = Rejected_epochs_B.get_data()
            Data_B_Rejected = Data_B_Rejected.reshape(Data_B_Rejected.shape[0], -1)
            Events_B_Rejected = Rejected_epochs_B.events[:, -1]
        else:
            Data_B_Rejected = 0
            Events_B_Rejected = 0
            print("Percentage of dropped epochs-B is 0")
            
        ##------------------------------------------------------------------------------------------
        
        # RSVP-Talk all epochs for testing 
        Data_T_All = Epoch_T_All.get_data()
        Data_T_All = Data_T_All.reshape(Data_T_All.shape[0], -1)
        Events_T_All = Epoch_T_All.events[:, -1]

        # RSVP-Talk clean epochs for testing
        if percentage_dropped_T < 80:
            Data_T_Clean = Clean_epochs_T.get_data()
            Data_T_Clean = Data_T_Clean.reshape(Data_T_Clean.shape[0], -1)
            Events_T_Clean = Clean_epochs_T.events[:, -1]
        else:
            Data_T_Clean = 0
            Events_T_Clean = 0
            print("Percentage of dropped epochs-T is greater than 80")
        
        # RSVP-Talk rejected epochs for testing
        if percentage_dropped_T > 0 :
            Data_T_Rejected = Rejected_epochs_T.get_data()
            Data_T_Rejected = Data_T_Rejected.reshape(Data_T_Rejected.shape[0], -1)
            Events_T_Rejected = Rejected_epochs_T.events[:, -1]
        else:
            Data_T_Rejected = 0
            Events_T_Rejected = 0
            print("Percentage of dropped epochs-T is 0")
        
        ##------------------------------------------------------------------------------------------
        
        # RSVP-Head all epochs for testing 
        Data_H_All = Epoch_H_All.get_data()
        Data_H_All = Data_H_All.reshape(Data_H_All.shape[0], -1)
        Events_H_All = Epoch_H_All.events[:, -1]

        # RSVP-Head clean epochs for testing
        if percentage_dropped_H < 80:
            Data_H_Clean = Clean_epochs_H.get_data()
            Data_H_Clean = Data_H_Clean.reshape(Data_H_Clean.shape[0], -1)
            Events_H_Clean = Clean_epochs_H.events[:, -1]
        else:
            Data_H_Clean = 0
            Events_H_Clean = 0
            print("Percentage of dropped epochs-H is greater than 80")
        
        # RSVP-Head rejected epochs for testing
        if percentage_dropped_H > 0 :
            Data_H_Rejected = Rejected_epochs_H.get_data()
            Data_H_Rejected = Data_H_Rejected.reshape(Data_H_Rejected.shape[0], -1)
            Events_H_Rejected = Rejected_epochs_H.events[:, -1]
        else:
            Data_H_Rejected = 0
            Events_H_Rejected = 0
            print("Percentage of dropped epochs-H is 0")
        
       ##----------------------------------------------------------------------------------

        ## Training

            best_params = {
            'alpha_1': 4.692680899768591e-06,
            'alpha_2': 3.0101214309175212e-05,
            'lambda_1': 1.3167456935454495e-05,
            'lambda_2': 9.129425537759534e-06
        }
        scaler = StandardScaler() # Normalize the features
        
        # Create BayesianRidge model
        mod = BayesianRidge(**best_params)
        
        # Create a pipeline with normalization and BayesianRidge
        clf = make_pipeline(scaler, mod)

        # Fit the model
        clf.fit(Data_C, Events_C)

        
       
        ##------------------------------------------------------------------------------------------     
        
        ## Testing Traditional RSVP

        ## Traditional RSVP (All epochs :- Ep3 and Ep6)
        y_pred_all_C = clf.predict(Data_C_All)
        rocauc_all_C= roc_auc_score(Events_C_All, y_pred_all_C)
        
        ## Traditional RSVP (Clean epochs)
        y_pred_clean_C = clf.predict(Data_C)
        rocauc_clean_C= roc_auc_score(Events_C, y_pred_clean_C)
        
        ## Traditional RSVP (Rejected epochs)
        if len(np.unique(Events_C_Rejected)) >1:
            y_pred_rej_C = clf.predict(Data_C_Rejected)
            rocauc_rej_C= roc_auc_score(Events_C_Rejected, y_pred_rej_C)
        else:
            rocauc_rej_C=0
            print("Error: There is only one class present")
        
        print(P_Num, 'RSVP-Traditional')
        ##------------------------------------------------------------------------------------------
        
        ## Testing Body RSVP
        
        ## Body RSVP (All epochs :- Ep3 and Ep6)
        y_pred_all_B = clf.predict(Data_B_All)
        rocauc_all_B= roc_auc_score(Events_B_All, y_pred_all_B)
        
        ## Body  RSVP (Clean epochs)
        if percentage_dropped_B < 80:
            y_pred_clean_B = clf.predict(Data_B_Clean)
            rocauc_clean_B= roc_auc_score(Events_B_Clean, y_pred_clean_B)
        else:
            rocauc_clean_B=0
            print("Cannot Classify: Percentage of dropped epochs-B is greater than 80")
        
        ## Body RSVP (Rejected epochs)
        if len(np.unique(Events_B_Rejected)) >1:
            y_pred_rej_B = clf.predict(Data_B_Rejected)
            rocauc_rej_B= roc_auc_score(Events_B_Rejected, y_pred_rej_B)

        else:
            rocauc_rej_B=0
            print("Cannot Classify: single class present")
        
        print(P_Num, 'RSVP-Body')
        ##------------------------------------------------------------------------------------------
        
        ## Testing Talking RSVP
        
        ## Talking RSVP (All epochs :- Ep3 and Ep6)
        y_pred_all_T = clf.predict(Data_T_All)
        rocauc_all_T= roc_auc_score(Events_T_All, y_pred_all_T)
        
        ## Talking RSVP (Clean epochs)
        if percentage_dropped_T < 80:
            y_pred_clean_T = clf.predict(Data_T_Clean)
            rocauc_clean_T= roc_auc_score(Events_T_Clean, y_pred_clean_T)
        else:
            rocauc_clean_T=0
            print("Cannot Classify: Percentage of dropped epochs-T is greater than 80")
        
        ## Talking RSVP (Rejected epochs)
        if len(np.unique(Events_T_Rejected)) >1:
            y_pred_rej_T = clf.predict(Data_T_Rejected)
            rocauc_rej_T= roc_auc_score(Events_T_Rejected, y_pred_rej_T)
                  
        else:
            rocauc_rej_T=0
            print("Cannot Classify: single class present")
        
        print(P_Num, 'RSVP-Taking')
        ##------------------------------------------------------------------------------------------
        
        ## Testing Head RSVP
        
        ## Head RSVP (All epochs :- Ep3 and Ep6)
        y_pred_all_H = clf.predict(Data_H_All)
        rocauc_all_H= roc_auc_score(Events_H_All, y_pred_all_H)
        
        ## Head  RSVP (Clean epochs)
        if percentage_dropped_H < 80:
            y_pred_clean_H = clf.predict(Data_H_Clean)
            rocauc_clean_H= roc_auc_score(Events_H_Clean, y_pred_clean_H)
        else:
            rocauc_clean_H=0
            print("Cannot Classify: Percentage of dropped epochs-H is greater than 80")
        
        ## Head RSVP (Rejected epochs)
        if len(np.unique(Events_H_Rejected)) >1:
            y_pred_rej_H = clf.predict(Data_H_Rejected)
            rocauc_rej_H= roc_auc_score(Events_H_Rejected, y_pred_rej_H)
        else:
            rocauc_rej_H=0
            print("Cannot Classify: single class present")                      

        
        print(P_Num, 'RSVP-Head')
       

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        X=00000000000.0 #  Just to add extrac column in csv file
        
        # Write the values to the CSV file
        writer.writerow([P_Num, round(rocauc_all_C, 2), round(rocauc_clean_C, 2), round(rocauc_rej_C, 2), X, 
                         round(rocauc_all_B, 2), round(rocauc_clean_B, 2), round(rocauc_rej_B, 2), X,
                        round(rocauc_all_T, 2), round(rocauc_clean_T, 2), round(rocauc_rej_T, 2), X,
                        round(rocauc_all_H, 2), round(rocauc_clean_H, 2), round(rocauc_rej_H, 2)])


**Table 3**

**Model trained and tested** on different conditions including only clean trials for training or testing and then training or testing the model using a combination of clean and bad trials

In [ ]:
import csv
THR=100e-6 # P2P threshold for trial rejection 
# Open a CSV file for writing
with open('RocAuc_TrainClean_&_TrainNoise_100uv_06Feb.csv', mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['P_Num', 'NIC-Clean', 'XXXXXXXXXXX', 'NIC-Clean+Noise', 'XXXXXXXXXXX', 'IC-Clean', 'XXXXXXXXXXX', 'IC-Clean+Noise', 'XXXXXXXXXXX', 'Train-IC-Clean-Test-NIC-Clean', 'XXXXXXXXXXX', 'Train-IC-Clean+Noise-Test-NIC-Clean+Noise'])

    for i in range(1, 11):
        P_Num = i
        
        
        
        #P_Num=10

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        def process_file(edf_file_path, csv_file_path, filename):
            raw=mne.io.read_raw_edf(edf_file_path, preload=True)

            # Pick 6 out of 32 channels
            #channel_names_to_select = ['Fz-CPz','Cz-CPz', 'P3-CPz', 'Pz-CPz', 'P4-CPz','Oz-CPz']
            #raw.pick_channels(channel_names_to_select)

            #Applying lowpass filter at 30Hz
            raw = raw.filter(.1, 30, method='iir')

            # Resampling data from 1000sps to 100sps
            raw = raw.resample(sfreq=100)


            #Re-Referencing the data to common average reference (CAR)
            raw= raw.set_eeg_reference(ref_channels='average')


            data = raw.get_data()
            Samples=len(data[0])

            annt=mne.events_from_annotations(raw, event_id='auto', regexp='^(?![Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$', use_rounding=True, chunk_duration=None, verbose=None)
            ann0=annt[0]
            Markers_Raw=ann0[:,0]
            L_annt_r=len(Markers_Raw)

            if (L_annt_r == 362):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end


            # Subject 5 -- Session 4 -- X1 :-
            ## This process is described and done in detail in another notebook.
            # there are total 371 markers instead of 362.
            # 1 marker from start and 9 markers from end are extra markers.
            # 1 Marker in at index 185 is also extra. We will remove it.

            if (L_annt_r == 371):
                Markers=Markers_Raw[1:-9] # :- Extras = 1 start and 9 end
                index1 = 185 # index_to_remove
                Markers = np.delete(Markers, index1)

            # Subject 7 -- Session 1 -- X1 :-
            ## This process is described and done in detail in another notebook.
            # there are total 371 markers instead of 362.
            # 1 marker from start and 1 marker from end are extra markers.
            # 2 Markers at index 116 and index 180 are missing. We will insert these markers.


            if (L_annt_r == 360):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end

                index1 = 116
                value1 = Markers[index1 - 1] + 25

                index2 = 180
                value2 = Markers[index2 - 1] + 25

                insert_indices = [index1, index2]
                values_to_insert = [value1, value2]

                Markers = np.insert(Markers, insert_indices, values_to_insert)    

            # Subject 8 -- Session 1 -- X4 :-
            ## This process is described and done in detail in another notebook.
            # there are total 363 markers instead of 362.
            # 1 marker from start and 9 markers from end are extra markers.
            # 1 Marker in at index 200 is also extra. We will remove it.

            if (L_annt_r == 363):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end
                index1 = 200 # index_to_remove
                Markers = np.delete(Markers, index1)

            # Subject 7 -- Session 1 -- X8 :-
            ## This process is described and done in detail in another notebook.
            # there are total 359 markers instead of 362.
            # 1 marker from start and 1 marker from end are extra markers.
            # 3 Markers at index 49, 50 and index 139 are missing. We will insert these markers.


            if (L_annt_r == 359):
                Markers=Markers_Raw[1:-1] # :- Extras = 1 start and 1 end

                index1 = 49
                value1 = Markers[index1 - 1] + 25

                index2 = 50
                value2 = Markers[index2 - 1] + 25

                index3 = 139
                value3 = Markers[index3 - 1] + 25

                insert_indices = [index1, index2, index3]
                values_to_insert = [value1, value2, value3]

                Markers = np.insert(Markers, insert_indices, values_to_insert)     



            else:
                print("Unknown length")
            L_annt0=len(Markers)
            print("Number of markers: ",L_annt0)

            Zeros = [0] * Samples
            Zero_arr = np.array(Zeros)
            Zero_2d = np.reshape(Zero_arr, (len(Zero_arr), 1))
            Zero_2dt=np.transpose(Zero_2d)

            #csv_path= file_path.replace('Data', 'Markers').replace('.edf', '.csv')
            csv_mark0 = pd.read_csv(csv_file_path, header=None)
            csv_mark = np.array(csv_mark0)

            zero_mark=Zero_arr
            csv1=0
            for x in Markers:   
                zero_mark[x]=csv_mark[csv1]
                csv1=csv1+1

            tar = np.where(zero_mark == 1)
            std = np.where(zero_mark == 2)

            new_events = []
            for st in std[0]:
                new_events.append([st, 0, 2]) 
            for t in tar[0]:
                new_events.append([t, 0, 1])

            event_dict = {'target':1, 'standard':2}
            epoch = mne.Epochs(raw, new_events, tmin=-0.2, tmax=0.8, event_id=event_dict,
                            preload=True, baseline=None)

            #----------------------------------------

            evoked_target = epoch['target'].average()
            evoked_standard = epoch['standard'].average()
            evoked_diff = mne.combine_evoked([evoked_target, evoked_standard], weights=[1, -1])
            #----------------------------------------------

     
            return epoch
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        # Only RSVP Tasks ( X1 and X2 )
        # Clean ---> c
        # There are two X1s and two X2s in each session

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_c1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X1-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_c1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X1-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c1 = os.path.basename(edf_path_c1).split('.')[0]
        epoch_c1 = process_file(edf_path_c1,csv_path_c1, filename_c1)



        edf_path_c2 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c2 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c2 = os.path.basename(edf_path_c2).split('.')[0]
        epoch_c2 = process_file(edf_path_c2,csv_path_c2, filename_c2)

        #----------------------------------------Session-2----------------------------------------

        edf_path_c3 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c3 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c3 = os.path.basename(edf_path_c3).split('.')[0]
        epoch_c3 = process_file(edf_path_c3,csv_path_c3, filename_c3)

        edf_path_c4 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c4 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c4 = os.path.basename(edf_path_c4).split('.')[0]
        epoch_c4 = process_file(edf_path_c4,csv_path_c4, filename_c4)

        #----------------------------------------Session-3----------------------------------------

        edf_path_c5 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c5 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c5 = os.path.basename(edf_path_c5).split('.')[0]
        epoch_c5 = process_file(edf_path_c5,csv_path_c5, filename_c5)

        edf_path_c6 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c6 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c6 = os.path.basename(edf_path_c6).split('.')[0]
        epoch_c6 = process_file(edf_path_c6,csv_path_c6, filename_c6)

        #----------------------------------------Session-4----------------------------------------

        edf_path_c7 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X1-eeg.edf'.format(P_Num, P_Num)
        csv_path_c7 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X1-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c7 = os.path.basename(edf_path_c7).split('.')[0]
        epoch_c7 = process_file(edf_path_c7,csv_path_c7, filename_c7)

        edf_path_c8 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X2-eeg.edf'.format(P_Num, P_Num)
        csv_path_c8 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X2-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_c8 = os.path.basename(edf_path_c8).split('.')[0]
        epoch_c8 = process_file(edf_path_c8,csv_path_c8, filename_c8)
 
        # Batch 1 training purposes
        Epoch_C = mne.concatenate_epochs([epoch_c1, epoch_c2, epoch_c3, epoch_c4, epoch_c5, epoch_c6, epoch_c7, epoch_c8])


        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        # RSVP with Body Movement ( X4 )
        # Body ---> b

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_b1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X4-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_b1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X4-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b1 = os.path.basename(edf_path_b1).split('.')[0]
        epoch_b1 = process_file(edf_path_b1,csv_path_b1, filename_b1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_b2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b2 = os.path.basename(edf_path_b2).split('.')[0]
        epoch_b2 = process_file(edf_path_b2,csv_path_b2, filename_b2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_b3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b3 = os.path.basename(edf_path_b3).split('.')[0]
        epoch_b3 = process_file(edf_path_b3,csv_path_b3, filename_b3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_b4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X4-eeg.edf'.format(P_Num, P_Num)
        csv_path_b4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X4-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_b4 = os.path.basename(edf_path_b4).split('.')[0]
        epoch_b4 = process_file(edf_path_b4,csv_path_b4, filename_b4)

        #------------------
        Epoch_B = mne.concatenate_epochs([epoch_b1, epoch_b2, epoch_b3, epoch_b4])

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------

        # RSVP with Talking ( X6 )
        # Talking ---> t

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_t1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X6-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_t1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X6-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t1 = os.path.basename(edf_path_t1).split('.')[0]
        epoch_t1 = process_file(edf_path_t1,csv_path_t1, filename_t1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_t2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t2 = os.path.basename(edf_path_t2).split('.')[0]
        epoch_t2 = process_file(edf_path_t2,csv_path_t2, filename_t2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_t3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t3 = os.path.basename(edf_path_t3).split('.')[0]
        epoch_t3 = process_file(edf_path_t3,csv_path_t3, filename_t3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_t4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X6-eeg.edf'.format(P_Num, P_Num)
        csv_path_t4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X6-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_t4 = os.path.basename(edf_path_t4).split('.')[0]
        epoch_t4 = process_file(edf_path_t4,csv_path_t4, filename_t4)

        #------------------
        Epoch_T = mne.concatenate_epochs([epoch_t1, epoch_t2, epoch_t3 ,epoch_t4])

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        # RSVP with Head Movement ( X8 )
        # Head ---> h

        #----------------------------------------Session-1----------------------------------------

        # Raw EEG edf file path
        edf_path_h1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Data\\P{}-Ss1-X8-eeg.edf'.format(P_Num, P_Num)

        # RSVP image labels file path
        csv_path_h1 = r'C:\\Users\\Data A\\P{}\\Session-1\\Mark\\P{}-Ss1-X8-mrk.csv'.format(P_Num, P_Num)

        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h1 = os.path.basename(edf_path_h1).split('.')[0]
        epoch_h1 = process_file(edf_path_h1,csv_path_h1, filename_h1)


        #----------------------------------------Session-2----------------------------------------

        edf_path_h2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Data\\P{}-Ss2-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h2 = r'C:\\Users\\Data A\\P{}\\Session-2\\Mark\\P{}-Ss2-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h2 = os.path.basename(edf_path_h2).split('.')[0]
        epoch_h2 = process_file(edf_path_h2,csv_path_h2, filename_h2)

        #----------------------------------------Session-3----------------------------------------

        edf_path_h3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Data\\P{}-Ss3-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h3 = r'C:\\Users\\Data A\\P{}\\Session-3\\Mark\\P{}-Ss3-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h3 = os.path.basename(edf_path_h3).split('.')[0]
        epoch_h3 = process_file(edf_path_h3,csv_path_h3, filename_h3)

        #----------------------------------------Session-4----------------------------------------

        edf_path_h4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Data\\P{}-Ss4-X8-eeg.edf'.format(P_Num, P_Num)
        csv_path_h4 = r'C:\\Users\\Data A\\P{}\\Session-4\\Mark\\P{}-Ss4-X8-mrk.csv'.format(P_Num, P_Num)
        #Processing the files to extract Epochs using the defined function 'process_file'
        filename_h4 = os.path.basename(edf_path_h4).split('.')[0]
        epoch_h4 = process_file(edf_path_h4,csv_path_h4, filename_h4)

        #------------------
        Epoch_H = mne.concatenate_epochs([epoch_h1,epoch_h2, epoch_h3, epoch_h4])
        
        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------       
        #THR=100e-6 #Threshold
        # Set your peak-to-peak threshold
        p2p_threshold_C = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_B = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_T = THR  # #you can change it.. may be 100e-6 as well
        p2p_threshold_H = THR  # #you can change it.. may be 100e-6 as well

        # Make the copy of epochs
        Epoch_C_All=Epoch_C.copy()
        Epoch_B_All=Epoch_B.copy()
        Epoch_T_All=Epoch_T.copy()
        Epoch_H_All=Epoch_H.copy()
        
        # Get the data from epochs
        Epoch_C_All_Data = Epoch_C_All.get_data()
        Epoch_B_All_Data = Epoch_B_All.get_data()
        Epoch_T_All_Data = Epoch_T_All.get_data()
        Epoch_H_All_Data = Epoch_H_All.get_data()
    

        # Calculate the peak-to-peak amplitude along the time axis
        p2p_amplitude_C = np.ptp(Epoch_C_All_Data, axis=2)
        p2p_amplitude_B = np.ptp(Epoch_B_All_Data, axis=2)
        p2p_amplitude_T = np.ptp(Epoch_T_All_Data, axis=2)
        p2p_amplitude_H = np.ptp(Epoch_H_All_Data, axis=2)
        

        # Find epochs that exceed the peak-to-peak threshold
        bad_epochs_C = np.where(p2p_amplitude_C > p2p_threshold_C)[0]
        bad_epochs_B = np.where(p2p_amplitude_B > p2p_threshold_B)[0]
        bad_epochs_T = np.where(p2p_amplitude_T > p2p_threshold_T)[0]
        bad_epochs_H = np.where(p2p_amplitude_H > p2p_threshold_H)[0]
        

        # Clean Epochs :- Drop bad epochs from the Epochs object
        Clean_epochs_C = Epoch_C_All.copy()
        Clean_epochs_C.drop(bad_epochs_C)
        
        Clean_epochs_B = Epoch_B_All.copy()
        Clean_epochs_B.drop(bad_epochs_B)
        
        Clean_epochs_T = Epoch_T_All.copy()
        Clean_epochs_T.drop(bad_epochs_T)
        
        Clean_epochs_H = Epoch_H_All.copy()
        Clean_epochs_H.drop(bad_epochs_H)

        # Rejected Epochs 
        
        Clean_indices_C = set(Clean_epochs_C.selection)                # Find the indices of clean epochs
        All_indices_C = set(Epoch_C_All.selection)                     # Find the indices of all epochs
        Rejected_indices_C = All_indices_C - Clean_indices_C           # Find the indices of rejected epochs
        Rejected_indices_C = list(Rejected_indices_C)                  # making a list of indices
        Rejected_indices_C = np.array(Rejected_indices_C).astype(int)  # Converting the list of indices to int
        Rejected_epochs_C = Epoch_C_All[Rejected_indices_C]
        
        Clean_indices_B = set(Clean_epochs_B.selection)                # Find the indices of clean epochs
        All_indices_B = set(Epoch_B_All.selection)                     # Find the indices of all epochs
        Rejected_indices_B = All_indices_B - Clean_indices_B           # Find the indices of rejected epochs
        Rejected_indices_B = list(Rejected_indices_B)                  # making a list of indices
        Rejected_indices_B = np.array(Rejected_indices_B).astype(int)  # Converting the list of indices to int
        Rejected_epochs_B = Epoch_B_All[Rejected_indices_B] 
        
        Clean_indices_T = set(Clean_epochs_T.selection)                # Find the indices of clean epochs
        All_indices_T = set(Epoch_T_All.selection)                     # Find the indices of all epochs
        Rejected_indices_T = All_indices_T - Clean_indices_T           # Find the indices of rejected epochs
        Rejected_indices_T = list(Rejected_indices_T)                  # making a list of indices
        Rejected_indices_T = np.array(Rejected_indices_T).astype(int)  # Converting the list of indices to int
        Rejected_epochs_T = Epoch_T_All[Rejected_indices_T]
        
        Clean_indices_H = set(Clean_epochs_H.selection)                # Find the indices of clean epochs
        All_indices_H = set(Epoch_H_All.selection)                     # Find the indices of all epochs
        Rejected_indices_H = All_indices_H - Clean_indices_H           # Find the indices of rejected epochs
        Rejected_indices_H = list(Rejected_indices_H)                  # making a list of indices
        Rejected_indices_H = np.array(Rejected_indices_H).astype(int)  # Converting the list of indices to int
        Rejected_epochs_H = Epoch_H_All[Rejected_indices_H]
        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------    
        
        # Getting the percentage of dropped/bad epochs

        percentage_dropped_C=(len(Rejected_epochs_C) / len(Epoch_C_All)) * 100
        
        percentage_dropped_B=(len(Rejected_epochs_B) / len(Epoch_B_All)) * 100
        
        percentage_dropped_T=(len(Rejected_epochs_T) / len(Epoch_T_All)) * 100   
        
        percentage_dropped_H=(len(Rejected_epochs_H) / len(Epoch_H_All)) * 100
        

        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 1--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on Traditional RSVP (Clean) and Test on Traditional RSVP (Clean)
        ##------------------------------------------------------------------------------------------

        Event_Clean_epochs_C = Clean_epochs_C.events
        standard_indices = np.where(Event_Clean_epochs_C[:, -1] == 2)[0]
        target_indices = np.where(Event_Clean_epochs_C[:, -1] == 1)[0]
        
        # Combine indices for both classes
        all_indices_Clean_epochs_C = np.concatenate([standard_indices, target_indices])
        
        # Fix the seed for reproducibility
        np.random.seed(42)
        
        # Randomly shuffle the combined indices
        np.random.shuffle(all_indices_Clean_epochs_C)
        
        # Select 1080 events for training
        num_training_events = 1080
        training_indices_Clean_epochs_C = all_indices_Clean_epochs_C[:num_training_events]
        
        # The remaining events will be used for testing
        testing_indices_Clean_epochs_C = all_indices_Clean_epochs_C[num_training_events:]
        
        # Separate Epoch_A into training and testing sets
        Clean_epochs_C_training = Clean_epochs_C[training_indices_Clean_epochs_C]
        
        Clean_epochs_C_testing = Clean_epochs_C[testing_indices_Clean_epochs_C]
        
        
        # Traditional RSVP Clean Epochs for Training
        Data_Clean_C_Training = Clean_epochs_C_training.get_data()
        Data_Clean_C_Training = Data_Clean_C_Training.reshape(Data_Clean_C_Training.shape[0], -1)
        Events_Clean_C_Training = Clean_epochs_C_training.events[:, -1]
        
        # Traditional RSVP Clean Epochs for Testing
        Data_Clean_C_Testing = Clean_epochs_C_testing.get_data()
        Data_Clean_C_Testing = Data_Clean_C_Testing.reshape(Data_Clean_C_Testing.shape[0], -1)
        Events_Clean_C_Testing = Clean_epochs_C_testing.events[:, -1]
        
        ## BayesianRidge :- Traditional RSVP
        
        scaler = StandardScaler() # Normalize the features
        
        # Create BayesianRidge model
        mod = BayesianRidge()
        
        # Create a pipeline with normalization and BayesianRidge
        clf1 = make_pipeline(scaler, mod)
        
        # Fit the model
        clf1.fit(Data_Clean_C_Training, Events_Clean_C_Training)
        
        y_pred_clean = clf1.predict(Data_Clean_C_Testing)
        rocauc_clean= roc_auc_score(Events_Clean_C_Testing, y_pred_clean)
        print(P_Num, " -Model Clean- ", round(rocauc_clean, 2))


        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 2--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on Traditional RSVP (Clean+Noise) and Test on Traditional RSVP (Clean+Noise)
        ##------------------------------------------------------------------------------------------

        Event_all_epochs_C=Epoch_C_All.events
        
        standard_indices = np.where(Event_all_epochs_C[:, -1] == 2)[0]
        len(standard_indices)
        
        target_indices = np.where(Event_all_epochs_C[:, -1] == 1)[0]
        len(target_indices)
        
        # Combine indices for both classes
        all_indices_all_epochs_C = np.concatenate([standard_indices, target_indices])
        
        # Fix the seed for reproducibility
        np.random.seed(42)
        
        # Randomly shuffle the combined indices
        np.random.shuffle(all_indices_all_epochs_C)
        
        # Select 1080 events for training
        num_training_events = 1080
        training_indices_all_epochs_C = all_indices_all_epochs_C[:num_training_events]
        
        # The remaining events will be used for testing
        testing_indices_all_epochs_C = all_indices_all_epochs_C[num_training_events:]
        
        # Separate Epoch_A into training and testing sets
        All_epochs_C_training = Epoch_C_All[training_indices_all_epochs_C]
        
        All_epochs_C_testing = Epoch_C_All[testing_indices_all_epochs_C]
    
            
        # Traditional RSVP Clean + Noise Epochs for Training
        Data_All_C_Training = All_epochs_C_training.get_data()
        Data_All_C_Training = Data_All_C_Training.reshape(Data_All_C_Training.shape[0], -1)
        Events_All_C_Training = All_epochs_C_training.events[:, -1]
        
        # Traditional RSVP Clean + Noise Epochs for Testing
        Data_All_C_Testing = All_epochs_C_testing.get_data()
        Data_All_C_Testing = Data_All_C_Testing.reshape(Data_All_C_Testing.shape[0], -1)
        Events_All_C_Testing = All_epochs_C_testing.events[:, -1]
    
        ## BayesianRidge :- Traditional RSVP
        
        scaler = StandardScaler() # Normalize the features
        
        # Create BayesianRidge model
        mod = BayesianRidge()
        
        # Create a pipeline with normalization and BayesianRidge
        clf2 = make_pipeline(scaler, mod)
        
        # Fit the model
        clf2.fit(Data_All_C_Training, Events_All_C_Training)
        
        y_pred_all = clf2.predict(Data_All_C_Testing)
        rocauc_all= roc_auc_score(Events_All_C_Testing, y_pred_all)
        print(P_Num, " -Model Clean+Noise- ", round(rocauc_all, 2))

        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 3--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on 3 Categories (Body+Talk+Head) Combined RSVP (Clean) and Test on 3 Categories (Body+Talk+Head) Combined RSVP (Clean)
        ##------------------------------------------------------------------------------------------

        if percentage_dropped_H < 90:
            Epoch_3Cat = mne.concatenate_epochs([Clean_epochs_H, Clean_epochs_B, Clean_epochs_T])
            # Make the copy of epochs
            Epoch_3Cat_Clean=Epoch_3Cat.copy()
               
            Event_clean_epochs_3Cat=Epoch_3Cat_Clean.events
            
            standard_indices = np.where(Event_clean_epochs_3Cat[:, -1] == 2)[0]
            len(standard_indices)
            
            target_indices = np.where(Event_clean_epochs_3Cat[:, -1] == 1)[0]
            len(target_indices)
            
            # Combine indices for both classes
            all_indices_clean_epochs_3Cat = np.concatenate([standard_indices, target_indices])
            
            # Fix the seed for reproducibility
            np.random.seed(42)
            
            # Randomly shuffle the combined indices
            np.random.shuffle(all_indices_clean_epochs_3Cat)
            
            # Select 1080 events for training
            num_training_events = 1080
            training_indices_clean_epochs_3Cat = all_indices_clean_epochs_3Cat[:num_training_events]
            
            # The remaining events will be used for testing
            testing_indices_clean_epochs_3Cat = all_indices_clean_epochs_3Cat[num_training_events:]
            
            # Separate Epoch_A into training and testing sets
            Clean_epochs_3Cat_training = Epoch_3Cat_Clean[training_indices_clean_epochs_3Cat]
            
            Clean_epochs_3Cat_testing = Epoch_3Cat_Clean[testing_indices_clean_epochs_3Cat]
        
            if len(Epoch_3Cat_Clean) < 1080:
                # Traditional RSVP Clean + Noise Epochs for Training
                Data_Clean_3Cat_Training = 0
                Data_Clean_3Cat_Training = 0
                Events_Clean_3Cat_Training = 0
                
                # Traditional RSVP Clean + Noise Epochs for Testing
                Data_Clean_3Cat_Testing = 0
                Data_Clean_3Cat_Testing = 0
                Events_Clean_3Cat_Testing = 0
    
                rocauc_3Cat_clean=0
                print("Cannot Classify: total number of clean trials are less than 1080")
    
            else:
                # Traditional RSVP Clean + Noise Epochs for Training
                Data_Clean_3Cat_Training = Clean_epochs_3Cat_training.get_data()
                Data_Clean_3Cat_Training = Data_Clean_3Cat_Training.reshape(Data_Clean_3Cat_Training.shape[0], -1)
                Events_Clean_3Cat_Training = Clean_epochs_3Cat_training.events[:, -1]
                
                # Traditional RSVP Clean + Noise Epochs for Testing
                Data_Clean_3Cat_Testing = Clean_epochs_3Cat_testing.get_data()
                Data_Clean_3Cat_Testing = Data_Clean_3Cat_Testing.reshape(Data_Clean_3Cat_Testing.shape[0], -1)
                Events_Clean_3Cat_Testing = Clean_epochs_3Cat_testing.events[:, -1]
        
                ## BayesianRidge :- Traditional RSVP
                
                scaler = StandardScaler() # Normalize the features
                
                # Create BayesianRidge model
                mod = BayesianRidge()
                
                # Create a pipeline with normalization and BayesianRidge
                clf3 = make_pipeline(scaler, mod)
                
                # Fit the model
                clf3.fit(Data_Clean_3Cat_Training, Events_Clean_3Cat_Training)
                
                y_pred_3Cat_clean = clf3.predict(Data_Clean_3Cat_Testing)
                rocauc_3Cat_clean= roc_auc_score(Events_Clean_3Cat_Testing, y_pred_3Cat_clean)
                print(P_Num, " -Model_3Cat_Clean- ", round(rocauc_3Cat_clean, 2))

        else:
            rocauc_3Cat_clean=0
            print("Cannot Classify: dropped percentage of head catergory is more than 90% ")

        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 4--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on 3 Categories (Body+Talk+Head) Combined RSVP (Clean+Noise) and Test on 3 Categories (Body+Talk+Head) Combined RSVP (Clean+Noise)
        ##------------------------------------------------------------------------------------------

        Epoch_3Cat = mne.concatenate_epochs([Epoch_H, Epoch_B, Epoch_T])
    
        # Make the copy of epochs
        Epoch_3Cat_All=Epoch_3Cat.copy()
    
        Event_all_epochs_3Cat=Epoch_3Cat_All.events
        
        standard_indices = np.where(Event_all_epochs_3Cat[:, -1] == 2)[0]
        len(standard_indices)
        
        target_indices = np.where(Event_all_epochs_3Cat[:, -1] == 1)[0]
        len(target_indices)
        
        # Combine indices for both classes
        all_indices_all_epochs_3Cat = np.concatenate([standard_indices, target_indices])
        
        # Fix the seed for reproducibility
        np.random.seed(42)
        
        # Randomly shuffle the combined indices
        np.random.shuffle(all_indices_all_epochs_3Cat)
        
        # Select 1080 events for training
        num_training_events = 1080
        training_indices_all_epochs_3Cat = all_indices_all_epochs_3Cat[:num_training_events]
        
        # The remaining events will be used for testing
        testing_indices_all_epochs_3Cat = all_indices_all_epochs_3Cat[num_training_events:]
        
        # Separate Epoch_A into training and testing sets
        All_epochs_3Cat_training = Epoch_3Cat_All[training_indices_all_epochs_3Cat]
        
        All_epochs_3Cat_testing = Epoch_3Cat_All[testing_indices_all_epochs_3Cat]
    
            
        # Traditional RSVP Clean + Noise Epochs for Training
        Data_All_3Cat_Training = All_epochs_3Cat_training.get_data()
        Data_All_3Cat_Training = Data_All_3Cat_Training.reshape(Data_All_3Cat_Training.shape[0], -1)
        Events_All_3Cat_Training = All_epochs_3Cat_training.events[:, -1]
        
        # Traditional RSVP Clean + Noise Epochs for Testing
        Data_All_3Cat_Testing = All_epochs_3Cat_testing.get_data()
        Data_All_3Cat_Testing = Data_All_3Cat_Testing.reshape(Data_All_3Cat_Testing.shape[0], -1)
        Events_All_3Cat_Testing = All_epochs_3Cat_testing.events[:, -1]
    
        ## BayesianRidge :- Traditional RSVP
        
        scaler = StandardScaler() # Normalize the features
        
        # Create BayesianRidge model
        mod = BayesianRidge()
        
        # Create a pipeline with normalization and BayesianRidge
        clf4 = make_pipeline(scaler, mod)
        
        # Fit the model
        clf4.fit(Data_All_3Cat_Training, Events_All_3Cat_Training)
        
        y_pred_3Cat_all = clf4.predict(Data_All_3Cat_Testing)
        rocauc_3Cat_all= roc_auc_score(Events_All_3Cat_Testing, y_pred_3Cat_all)
        print(P_Num, " -Model_3Cat_Clean+Noise- ", round(rocauc_3Cat_all, 2))

        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 5--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on 3 Categories (Body+Talk+Head) Combined RSVP (Clean+Noise) and Test on Traditional RSVP (Clean)
        ##------------------------------------------------------------------------------------------

        if percentage_dropped_H < 90:
            Epoch_3Cat = mne.concatenate_epochs([Clean_epochs_H, Clean_epochs_B, Clean_epochs_T])
            # Make the copy of epochs
            Epoch_3Cat_Clean=Epoch_3Cat.copy()
               
            Event_clean_epochs_3Cat=Epoch_3Cat_Clean.events
            
            standard_indices = np.where(Event_clean_epochs_3Cat[:, -1] == 2)[0]
            len(standard_indices)
            
            target_indices = np.where(Event_clean_epochs_3Cat[:, -1] == 1)[0]
            len(target_indices)
            
            # Combine indices for both classes
            all_indices_clean_epochs_3Cat = np.concatenate([standard_indices, target_indices])
            
            # Fix the seed for reproducibility
            np.random.seed(42)
            
            # Randomly shuffle the combined indices
            np.random.shuffle(all_indices_clean_epochs_3Cat)
            
            # Select 1080 events for training
            num_training_events = 1080
            training_indices_clean_epochs_3Cat = all_indices_clean_epochs_3Cat[:num_training_events] 
           
            # Separate Epoch_A into training and testing sets
            Clean_epochs_3Cat_training = Epoch_3Cat_Clean[training_indices_clean_epochs_3Cat]
            
        
            if len(Epoch_3Cat_Clean) < 1080:
                # Traditional RSVP Clean + Noise Epochs for Training
                Data_Clean_3Cat_Training = 0
                Data_Clean_3Cat_Training = 0
                Events_Clean_3Cat_Training = 0
                
                # Traditional RSVP Clean + Noise Epochs for Testing
                Data_Clean_3Cat_Testing = 0
                Data_Clean_3Cat_Testing = 0
                Events_Clean_3Cat_Testing = 0
    
                rocauc_3Cat_clean=0
                print("Cannot Classify: total number of clean trials are less than 1080")
    
            else:
                # Traditional RSVP Clean + Noise Epochs for Training
                Data_Clean_3Cat_Training = Clean_epochs_3Cat_training.get_data()
                Data_Clean_3Cat_Training = Data_Clean_3Cat_Training.reshape(Data_Clean_3Cat_Training.shape[0], -1)
                Events_Clean_3Cat_Training = Clean_epochs_3Cat_training.events[:, -1] 
                
                # Traditional RSVP (Clean) Epochs for Testing
                Data_Clean_C_Testing = Clean_epochs_C_testing.get_data()
                Data_Clean_C_Testing = Data_Clean_C_Testing.reshape(Data_Clean_C_Testing.shape[0], -1)
                Events_Clean_C_Testing = Clean_epochs_C_testing.events[:, -1]
            
                ## BayesianRidge :- Traditional RSVP
                
                scaler = StandardScaler() # Normalize the features
                
                # Create BayesianRidge model
                mod = BayesianRidge()
                
                # Create a pipeline with normalization and BayesianRidge
                clf5 = make_pipeline(scaler, mod)
                
                # Fit the model
                clf5.fit(Data_Clean_3Cat_Training, Events_Clean_3Cat_Training)
                
                y_pred_Tr3Cat_TsC = clf5.predict(Data_Clean_C_Testing)
                rocauc_Tr3Cat_TsC= roc_auc_score(Events_Clean_C_Testing, y_pred_Tr3Cat_TsC)
                print(P_Num, " -Model_Tr3Cat_Clean_TsClean- ", round(rocauc_Tr3Cat_TsC, 2))

        else:
            rocauc_Tr3Cat_TsC=0
            print("Cannot Classify: dropped percentage of head catergory is more than 90% ")
        
        ##------------------------------------------------------------------------------------------
        ##----------------------------------------Part 6--------------------------------------------
        ##------------------------------------------------------------------------------------------
        ## Train on 3 Categories (Body+Talk+Head) Combined RSVP (Clean+Noise) and Test on Traditional RSVP (Clean+Noise)
        ##------------------------------------------------------------------------------------------

        Epoch_3Cat = mne.concatenate_epochs([Epoch_H, Epoch_B, Epoch_T])
    
        # Make the copy of epochs
        Epoch_3Cat_All=Epoch_3Cat.copy()
    
        Event_all_epochs_3Cat=Epoch_3Cat_All.events
        
        standard_indices = np.where(Event_all_epochs_3Cat[:, -1] == 2)[0]
        len(standard_indices)
        
        target_indices = np.where(Event_all_epochs_3Cat[:, -1] == 1)[0]
        len(target_indices)
        
        # Combine indices for both classes
        all_indices_all_epochs_3Cat = np.concatenate([standard_indices, target_indices])
        
        # Fix the seed for reproducibility
        np.random.seed(42)
        
        # Randomly shuffle the combined indices
        np.random.shuffle(all_indices_all_epochs_3Cat)
        
        # Select 1080 events for training
        num_training_events = 1080
        training_indices_all_epochs_3Cat = all_indices_all_epochs_3Cat[:num_training_events]
        
        
        # Separate Epoch_A into training and testing sets
        All_epochs_3Cat_training = Epoch_3Cat_All[training_indices_all_epochs_3Cat]
        
        
            
        # Artifactual RSVP (Clean + Noise) Epochs for Training
        Data_All_3Cat_Training = All_epochs_3Cat_training.get_data()
        Data_All_3Cat_Training = Data_All_3Cat_Training.reshape(Data_All_3Cat_Training.shape[0], -1)
        Events_All_3Cat_Training = All_epochs_3Cat_training.events[:, -1]
        
        # Traditional RSVP (Clean+Bad) Epochs for Testing
        Data_All_C_Testing = All_epochs_C_testing.get_data()
        Data_All_C_Testing = Data_All_C_Testing.reshape(Data_All_C_Testing.shape[0], -1)
        Events_All_C_Testing = All_epochs_C_testing.events[:, -1]
    
        ## BayesianRidge :- Traditional RSVP
        
        scaler = StandardScaler() # Normalize the features
        
        # Create BayesianRidge model
        mod = BayesianRidge()
        
        # Create a pipeline with normalization and BayesianRidge
        clf6 = make_pipeline(scaler, mod)
        
        # Fit the model
        clf6.fit(Data_All_3Cat_Training, Events_All_3Cat_Training)
        
        y_pred_Tr3Cat_TsCB = clf6.predict(Data_All_C_Testing)
        rocauc_Tr3Cat_TsCB= roc_auc_score(Events_All_C_Testing, y_pred_Tr3Cat_TsCB)
        print(P_Num, " -Model_Tr3Cat_Clean+Noise_TsCleanBad- ", round(rocauc_Tr3Cat_TsCB, 2))

        
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------
        ##------------------------------------------------------------------------------------------


        

        X=00000000000.0 #  Just to add extrac column in csv file
        
        # Write the values to the CSV file
        writer.writerow([P_Num, round(rocauc_clean, 2), X, round(rocauc_all, 2), X, round(rocauc_3Cat_clean, 2), X, round(rocauc_3Cat_all, 2), X, round(rocauc_Tr3Cat_TsC, 2), X, round(rocauc_Tr3Cat_TsCB, 2)])
